In [1]:
from openai import OpenAI
client = OpenAI()

In [2]:
from typing_extensions import override
from openai import AssistantEventHandler
import base64
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  
  def __init__(self):
    self.text_delta_values = ''
    self.image_data = ''
    self.image_id = ''
    super().__init__()
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
    self.text_delta_values+=delta.value
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          print(output.type)
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
          elif output.type == "image":
            print(type(output.image))
            image_id = output.image.file_id
            print(image_id)
            self.image_id = image_id
            image_data = client.files.content(image_id)
            image_data_bytes = image_data.read()
            self.image_data = base64.b64encode(image_data_bytes).decode('utf-8')
            with open(f"./image-{image_id}.png", "wb") as file:
                file.write(image_data_bytes)

In [5]:
thread = client.beta.threads.create()
message_content = """
What day of the week were the most bookings made? Generate code to perform the calculations to answer. Then, create a visualization using seaborn. 
'file-TzOVAHko5TwyrRsaPySmIR0Z' and 'file-4FKpqlvJOZqxNUfIJHAHRGJY' are '.csv' files containing data
'file-LbOzhUfzfTVqYNRUOPderfwy' is a '.txt' containing data information about what each column represents in the csv data files above
"""

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content= message_content
)

event_handler = EventHandler()

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id="asst_gwH0YPXzLZpaUoNnGQzmkOaG", # asst_EKZ2miOCbP7AayhZLCSzhkQK OR  asst_gwH0YPXzLZpaUoNnGQzmkOaG (3)
  instructions="Answer with minimal text and output image data",
  event_handler=event_handler,
) as stream:
  stream.until_done()



assistant > First, I will load and examine the contents of the '.csv' files to determine the structure of the data and find the relevant columns for booking dates. Then, I'll perform the necessary calculations to find the day of the week when the most bookings were made and create a visualization using seaborn. Let's begin by loading the '.csv' files.
assistant > code_interpreter

import pandas as pd

# Load the CSV files
file_path_1 = '/mnt/data/file-TzOVAHko5TwyrRsaPySmIR0Z'
file_path_2 = '/mnt/data/file-4FKpqlvJOZqxNUfIJHAHRGJY'

data_1 = pd.read_csv(file_path_1)
data_2 = pd.read_csv(file_path_2)

data_1.head(), data_2.head()

output >
logs

(           ds                               id_user  ds_checkin ds_checkout  \
 0  2014-10-01  0000af0a-6f26-4233-9832-27efbfb36148  2014-10-09  2014-10-12   
 1  2014-10-01  0000af0a-6f26-4233-9832-27efbfb36148  2014-10-09  2014-10-19   
 2  2014-10-01  000cd9d3-e05b-4016-9e09-34a6f8ba2fc5         NaN         NaN   
 3  2014-10-01  000cd9d3-e

In [9]:
# event_handler now holds all the data needed
# print(f"{event_handler.text_delta_values}\n{event_handler.image_id}")

prompt = f"""
Extract the answer to the user's prompt from the text. 

User prompt: '{message_content}'
Text: '{event_handler.text_delta_values}'

"""
response =  client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "user", "content": prompt}],
)

print(response.choices[0].message.content.strip())
print(event_handler.image_data)


The day of the week with the most bookings is Monday with a total of 363 bookings.
...


<h1>Actual Wrapper</h1>